In [ ]:
###
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
# Any results you write to the current directory are saved as output.

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import category_encoders as ce
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm_notebook as tqdm

In [ ]:
df_train = pd.read_csv('../input/train.csv',index_col = 0)
df_test = pd.read_csv('../input/test.csv',index_col = 0)
df_country_info = pd.read_csv('../input/country_info.csv',index_col = 0)

In [ ]:
df_country_info.dtypes

In [ ]:
df_country_info.replace(',', '')

In [ ]:
df_country_info.head()

In [ ]:
df_train = pd.merge(df_train, df_country_info, on='Country')
df_test = pd.merge(df_test, df_country_info, on='Country')

In [ ]:
df_train.head()

In [ ]:
#col = ['AssessJob1','AssessJob2','AssessJob3','AssessJob4','AssessJob5','AssessJob6','AssessJob7','AssessJob8','AssessJob9','AssessJob10','AssessBenefits1','AssessBenefits2','AssessBenefits3','AssessBenefits4','AssessBenefits5','AssessBenefits6','AssessBenefits7','AssessBenefits8','AssessBenefits9','AssessBenefits10','AssessBenefits11','JobContactPriorities1','JobContactPriorities2','JobContactPriorities3','JobContactPriorities4','JobContactPriorities5','JobEmailPriorities1','JobEmailPriorities2','JobEmailPriorities3','JobEmailPriorities4','JobEmailPriorities5','JobEmailPriorities6','JobEmailPriorities7','AdsPriorities1','AdsPriorities2','AdsPriorities3','AdsPriorities4','AdsPriorities5','AdsPriorities6','AdsPriorities7','Hobby','OpenSource','Country','Student','Employment','FormalEducation','UndergradMajor','CompanySize','DevType','YearsCoding','YearsCodingProf','JobSatisfaction','CareerSatisfaction','HopeFiveYears','JobSearchStatus','LastNewJob','UpdateCV','Currency','SalaryType','CurrencySymbol','CommunicationTools','TimeFullyProductive','TimeAfterBootcamp','AgreeDisagree1','AgreeDisagree2','AgreeDisagree3','FrameworkWorkedWith','OperatingSystem','NumberMonitors','CheckInCode','AdBlocker','AdBlockerDisable','AdsAgreeDisagree1','AdsAgreeDisagree2','AdsAgreeDisagree3','AdsActions','AIDangerous','AIInteresting','AIResponsible','AIFuture','EthicsChoice','EthicsReport','EthicsResponsible','EthicalImplications','StackOverflowRecommend','StackOverflowVisit','StackOverflowHasAccount','StackOverflowParticipate','StackOverflowJobs','StackOverflowDevStory','StackOverflowJobsRecommend','StackOverflowConsiderMember','HypotheticalTools1','HypotheticalTools2','HypotheticalTools3','HypotheticalTools4','HypotheticalTools5','WakeTime','HoursComputer','HoursOutside','SkipMeals','ErgonomicDevices','Exercise','Gender','SexualOrientation','EducationParents','RaceEthnicity','Age','Dependents','MilitaryUS','SurveyTooLong','SurveyEasy']
nums = ['Population','Area (sq. mi.)','GDP ($ per capita)']
cats  = ['Hobby','OpenSource','Country','Student','Employment','FormalEducation','UndergradMajor','CompanySize','DevType','NumberMonitors','YearsCoding','YearsCodingProf','JobSatisfaction','CareerSatisfaction','HopeFiveYears','JobSearchStatus','LastNewJob','AssessJob1','AssessJob2','AssessJob3','AssessJob4','AssessJob5','AssessJob6','AssessJob7','AssessJob8','AssessJob9','AssessJob10','AssessBenefits1','AssessBenefits2','AssessBenefits3','AssessBenefits4','AssessBenefits5','AssessBenefits6','AssessBenefits7','AssessBenefits8','AssessBenefits9','AssessBenefits10','AssessBenefits11','JobContactPriorities1','JobContactPriorities2','JobContactPriorities3','JobContactPriorities4','JobContactPriorities5','JobEmailPriorities1','JobEmailPriorities2','JobEmailPriorities3','JobEmailPriorities4','JobEmailPriorities5','JobEmailPriorities6','JobEmailPriorities7','UpdateCV','Currency','SalaryType','CurrencySymbol','CommunicationTools','TimeFullyProductive','TimeAfterBootcamp','AgreeDisagree1','AgreeDisagree2','AgreeDisagree3','FrameworkWorkedWith','OperatingSystem','CheckInCode','AdBlocker','AdBlockerDisable','AdsAgreeDisagree1','AdsAgreeDisagree2','AdsAgreeDisagree3','AdsActions','AdsPriorities1','AdsPriorities2','AdsPriorities3','AdsPriorities4','AdsPriorities5','AdsPriorities6','AdsPriorities7','AIDangerous','AIInteresting','AIResponsible','AIFuture','EthicsChoice','EthicsReport','EthicsResponsible','EthicalImplications','StackOverflowRecommend','StackOverflowVisit','StackOverflowHasAccount','StackOverflowParticipate','StackOverflowJobs','StackOverflowDevStory','StackOverflowJobsRecommend','StackOverflowConsiderMember','HypotheticalTools1','HypotheticalTools2','HypotheticalTools3','HypotheticalTools4','HypotheticalTools5','WakeTime','HoursComputer','HoursOutside','SkipMeals','ErgonomicDevices','Exercise','Gender','SexualOrientation','EducationParents','RaceEthnicity','Age','Dependents','MilitaryUS','SurveyTooLong','SurveyEasy']

df_train=df_train.drop(['Region','Pop. Density (per sq. mi.)','Coastline (coast/area ratio)','Net migration','Infant mortality (per 1000 births)','Literacy (%)','Phones (per 1000)','Arable (%)','Crops (%)','Other (%)','Climate','Birthrate','Deathrate','Agriculture','Industry','Service'],axis=1)
df_test=df_test.drop(['Region','Pop. Density (per sq. mi.)','Coastline (coast/area ratio)','Net migration','Infant mortality (per 1000 births)','Literacy (%)','Phones (per 1000)','Arable (%)','Crops (%)','Other (%)','Climate','Birthrate','Deathrate','Agriculture','Industry','Service'],axis=1)

#drop_col = ['Hobby','OpenSource','Country','Student','Employment','FormalEducation','UndergradMajor','CompanySize','DevType','YearsCoding','YearsCodingProf','JobSatisfaction','CareerSatisfaction','HopeFiveYears','JobSearchStatus','LastNewJob','UpdateCV','Currency','SalaryType','CurrencySymbol','CommunicationTools','TimeFullyProductive','TimeAfterBootcamp','AgreeDisagree1','AgreeDisagree2','AgreeDisagree3','FrameworkWorkedWith','OperatingSystem','NumberMonitors','CheckInCode','AdBlocker','AdBlockerDisable','AdsAgreeDisagree1','AdsAgreeDisagree2','AdsAgreeDisagree3','AdsActions','AIDangerous','AIInteresting','AIResponsible','AIFuture','EthicsChoice','EthicsReport','EthicsResponsible','EthicalImplications','StackOverflowRecommend','StackOverflowVisit','StackOverflowHasAccount','StackOverflowParticipate','StackOverflowJobs','StackOverflowDevStory','StackOverflowJobsRecommend','StackOverflowConsiderMember','HypotheticalTools1','HypotheticalTools2','HypotheticalTools3','HypotheticalTools4','HypotheticalTools5','WakeTime','HoursComputer','HoursOutside','SkipMeals','ErgonomicDevices','Exercise','Gender','SexualOrientation','EducationParents','RaceEthnicity','Age','Dependents','MilitaryUS','SurveyTooLong','SurveyEasy']
#df_train = df_train.drop(['Hobby','OpenSource','Country','Student','Employment','FormalEducation','UndergradMajor','CompanySize','DevType','YearsCoding','YearsCodingProf','JobSatisfaction','CareerSatisfaction','HopeFiveYears','JobSearchStatus','LastNewJob','UpdateCV','Currency','SalaryType','CurrencySymbol','CommunicationTools','TimeFullyProductive','TimeAfterBootcamp','AgreeDisagree1','AgreeDisagree2','AgreeDisagree3','FrameworkWorkedWith','OperatingSystem','NumberMonitors','CheckInCode','AdBlocker','AdBlockerDisable','AdsAgreeDisagree1','AdsAgreeDisagree2','AdsAgreeDisagree3','AdsActions','AIDangerous','AIInteresting','AIResponsible','AIFuture','EthicsChoice','EthicsReport','EthicsResponsible','EthicalImplications','StackOverflowRecommend','StackOverflowVisit','StackOverflowHasAccount','StackOverflowParticipate','StackOverflowJobs','StackOverflowDevStory','StackOverflowJobsRecommend','StackOverflowConsiderMember','HypotheticalTools1','HypotheticalTools2','HypotheticalTools3','HypotheticalTools4','HypotheticalTools5','WakeTime','HoursComputer','HoursOutside','SkipMeals','ErgonomicDevices','Exercise','Gender','SexualOrientation','EducationParents','RaceEthnicity','Age','Dependents','MilitaryUS','SurveyTooLong','SurveyEasy'],axis = 1)
#df_test = df_test.drop(['Hobby','OpenSource','Country','Student','Employment','FormalEducation','UndergradMajor','CompanySize','DevType','YearsCoding','YearsCodingProf','JobSatisfaction','CareerSatisfaction','HopeFiveYears','JobSearchStatus','LastNewJob','UpdateCV','Currency','SalaryType','CurrencySymbol','CommunicationTools','TimeFullyProductive','TimeAfterBootcamp','AgreeDisagree1','AgreeDisagree2','AgreeDisagree3','FrameworkWorkedWith','OperatingSystem','NumberMonitors','CheckInCode','AdBlocker','AdBlockerDisable','AdsAgreeDisagree1','AdsAgreeDisagree2','AdsAgreeDisagree3','AdsActions','AIDangerous','AIInteresting','AIResponsible','AIFuture','EthicsChoice','EthicsReport','EthicsResponsible','EthicalImplications','StackOverflowRecommend','StackOverflowVisit','StackOverflowHasAccount','StackOverflowParticipate','StackOverflowJobs','StackOverflowDevStory','StackOverflowJobsRecommend','StackOverflowConsiderMember','HypotheticalTools1','HypotheticalTools2','HypotheticalTools3','HypotheticalTools4','HypotheticalTools5','WakeTime','HoursComputer','HoursOutside','SkipMeals','ErgonomicDevices','Exercise','Gender','SexualOrientation','EducationParents','RaceEthnicity','Age','Dependents','MilitaryUS','SurveyTooLong','SurveyEasy'],axis = 1)

In [ ]:
df_train.dtypes

In [ ]:
df_train[nums] = df_train[nums].fillna(df_train[nums].mean())
df_test[nums] = df_test[nums].fillna(df_test[nums].mean())

df_train[cats] = df_train[cats].fillna("NaN")
df_test[cats] = df_test[cats].fillna("NaN")

In [ ]:
df_train['ConvertedSalary'] = df_train['ConvertedSalary'].astype(np.int64)
y_train = df_train['ConvertedSalary']
#x_train = df_train.drop(['loan_condition','issue_d','emp_title','emp_length','mths_since_last_delinq','mths_since_last_record','mths_since_last_major_derog','tot_coll_amt','tot_cur_bal','Latitude','Longitude','addr_state'],axis=1)
x_train = df_train.drop(['ConvertedSalary'],axis = 1)
x_test = df_test
#x_test = df_test.drop(['issue_d','emp_title','emp_length','mths_since_last_delinq','mths_since_last_record','mths_since_last_major_derog','tot_coll_amt','tot_cur_bal','Latitude','Longitude','addr_state'],axis=1)

In [ ]:
x_train[nums].head()

In [ ]:
scaler = StandardScaler()
scaler.fit(x_train[nums])
x_train[nums] = scaler.transform(x_train[nums])
x_test[nums] = scaler.transform(x_test[nums])

In [ ]:
oe = ce.OrdinalEncoder(cols = cats,return_df=False)

x_train[cats] = oe.fit_transform(x_train[cats])
x_test[cats] = oe.transform(x_test[cats])

In [ ]:
#トレーニングデータとバリデーションデータに分割
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.33, random_state=71)

In [ ]:
import numpy as np
import lightgbm as lgb
from sklearn.datasets import load_boston
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold, train_test_split

# create dataset for lightgbm
lgb_train = lgb.Dataset(x_train, y_train)
lgb_eval = lgb.Dataset(x_val, y_val, reference=lgb_train)
# LightGBM parameters
params = {
        'task' : 'train',
        'boosting_type' : 'gbdt',
        'objective' : 'regression',
        'metric' : {'l2'},
        'num_leaves' : 31,
        'learning_rate' : 0.1,
        'feature_fraction' : 0.9,
        'bagging_fraction' : 0.8,
        'bagging_freq': 5,
        'verbose' : 0
}

# train
gbm = lgb.train(params,
            lgb_train,
            num_boost_round=200,
            valid_sets=lgb_eval,
            early_stopping_rounds=500)
y_pred = gbm.predict(x_val, num_iteration=gbm.best_iteration)

In [ ]:
 # テストデータを予測する
    y_pred = gbm.predict(x_test, num_iteration=gbm.best_iteration)

In [ ]:
submission = pd.read_csv('../input/sample_submission.csv', index_col=0)
submission.ConvertedSalary = y_pred
submission.to_csv('submission.csv')

In [ ]:
nan